#### Import Necessary Library

In [1]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq   # Or ChatOpenAI if you prefer OpenAI
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.documents import Document
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
#from langchain_core.output_parsers import StrOutputParser
#from langchain_community.llms import Ollama

In [ ]:
## Langsmith Tracking
#os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
#os.environ["LANGCHAIN_TRACING_V2"]="true"
#os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
# set embedding tech
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

g:\Gen AI-Krish Naik\5. Personal Gen AI Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
# load the data and fetch the index path from faiss db
#PDF_PATH = "Query_Data.pdf"
#INDEX_PATH = "faiss_index"

In [ ]:
#from langchain_community.vectorstores import FAISS
# load from disk
#new_db = FAISS.load_local(INDEX_PATH,embeddings,allow_dangerous_deserialization=True)

In [3]:
# load from disk -chromadb
from langchain_chroma import Chroma
new_db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [4]:
query="Do you support Facebook Messenger?"
docs=new_db.similarity_search(query)
docs

[Document(id='697d5e50-22cc-44d9-92ba-be58eae49886', metadata={}, page_content="Q: How do I reset my password?\nA: Click on 'Forgot Password' on the login page and follow the steps.\nQ: What should I do if my laptop doesn’t turn on?\nA: Please check the power connection and try holding the power button for 10 seconds.\nQ: My payment failed but money deducted.\nA: Please provide your transaction ID. We will check and process a refund if applicable.")]

In [6]:
# retriever object to query similar documents
retriever = new_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000224C4F17BF0>, search_kwargs={'k': 3})

In [ ]:
# %% 2. Define LLM

In [7]:
import os
from dotenv import load_dotenv
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [8]:
load_dotenv()
## load the GROQ API Key
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000224C8501430>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000224C873C830>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [35]:
## Prompt Template
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [14]:
## Document chain for creation-custom prompt for answering questions with context.

document_chain =  create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000224C8501430>, async_client=<groq.resources.chat.completions.AsyncCompletio

In [15]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Can you integrate with Salesforce?",
    "context":[Document(page_content="Q: How do I reset my password?\nA: Click on 'Forgot Password' on the login page and follow the steps.\nQ: What should I do if my laptop doesn’t turn on?\nA: Please check the power connection and try holding the power button for 10 seconds.\nQ: My payment failed but money deducted.\nA: Please provide your transaction ID. We will check and process a refund if applicable.")]
})

"I don't know."

In [16]:
### Input--->Retriever--->vectorstoredb
new_db

In [17]:
# %% 3. Conversational Retriever Setup
retriever = new_db.as_retriever()

from langchain.chains import create_retrieval_chain

Rag_chains = create_retrieval_chain(retriever,document_chain)


In [18]:
Rag_chains

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000224C4F17BF0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know th

In [19]:
## Get the response form the LLM
response=Rag_chains.invoke({"input":"Who manages payroll queries?"})
response['answer']

"I don't know."

### User and AI Chat History


In [20]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Prompt template for conversation

retriever_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. "
    "Rewrite the user question into a standalone query using chat history if needed."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [21]:
# History-aware retriever (rephrases user queries based on conversation)
history_aware_retriever = create_history_aware_retriever( llm, retriever, retriever_prompt )
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000224C4F17BF0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [22]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [23]:
# Full retrieval chain (retriever + doc_chain)
new_document_chain = create_stuff_documents_chain(llm,qa_prompt)
new_rag_retriever_chain = create_retrieval_chain(history_aware_retriever,new_document_chain)
new_rag_retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000224C4F17BF0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.An

#### chat conversation history store

In [24]:
# manual chat history store
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="Can I chat on WhatsApp?"
response1=new_rag_retriever_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="How do I update my billing information?"
response2=new_rag_retriever_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

I don't know. The context does not mention WhatsApp.


In [25]:
chat_history

[HumanMessage(content='Can I chat on WhatsApp?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I don't know. The context does not mention WhatsApp.", additional_kwargs={}, response_metadata={})]

In [26]:
# automatically store all chat history 
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Wrap your RAG chain with history support
conversational_rag_chain = RunnableWithMessageHistory(
    new_rag_retriever_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [27]:
result = conversational_rag_chain.invoke(
        {"input": "How do I update my billing information? "},
        config={"configurable": {"session_id": "user1"}}
    )

In [28]:
result

{'input': 'How do I update my billing information? ',
 'chat_history': [],
 'context': [Document(id='697d5e50-22cc-44d9-92ba-be58eae49886', metadata={}, page_content="Q: How do I reset my password?\nA: Click on 'Forgot Password' on the login page and follow the steps.\nQ: What should I do if my laptop doesn’t turn on?\nA: Please check the power connection and try holding the power button for 10 seconds.\nQ: My payment failed but money deducted.\nA: Please provide your transaction ID. We will check and process a refund if applicable.")],
 'answer': "I don't know. The given context does not provide information on updating billing information."}

In [29]:
def ask_with_history(session_id: str, question: str):
    """
    Ask a question with multi-user chat history support.
    Each session_id has its own stored conversation.
    """
    response = conversational_rag_chain.invoke(
        {"input": question},
        config={"configurable": {"session_id": session_id}}
    )
    return response["answer"]

In [30]:
# User 1 asks questions
print("User1:", ask_with_history("user1", "How do I update my billing information? "))

# User 2 starts a different session
print("User2:", ask_with_history("user2", "My product needs an update"))

# Check history for user1
history1 = get_session_history("user1")
print("\nUser1 Chat History:")
for msg in history1.messages:
    print(f"{msg.type.upper()}: {msg.content}")

User1: Unfortunately, I don't have the information on how to update billing information in the given context.
User2: I don't know.

User1 Chat History:
HUMAN: How do I update my billing information? 
AI: I don't know. The given context does not provide information on updating billing information.
HUMAN: How do I update my billing information? 
AI: Unfortunately, I don't have the information on how to update billing information in the given context.


In [31]:
print("User1:", ask_with_history("user1", "What should I do if my laptop doesn’t turn on?"))

User1: Please check the power connection and try holding the power button for 10 seconds.


In [37]:

print("User1:", ask_with_history("user1", "What next"))

User1: If holding the power button for 10 seconds doesn't work, you should try checking if the power cord is securely plugged in and if the outlet is working properly.


In [39]:
print("User1:", ask_with_history("user1", "What next"))

User1: I didn't provide any previous information to continue from. If you'd like to ask another question or provide more context, I'd be happy to help.


In [41]:
print(chat_history)

[HumanMessage(content='Can I chat on WhatsApp?', additional_kwargs={}, response_metadata={}), AIMessage(content="I don't know. The context does not mention WhatsApp.", additional_kwargs={}, response_metadata={})]


#### Other Approach

In [44]:
# load from disk -chromadb
from langchain_chroma import Chroma
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
retriever = vectorstore.as_retriever()

In [45]:
llm2=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")
llm2

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000224C8FED040>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000224C8FECDA0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [46]:
retriever_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Rewrite the user’s question into a clear, standalone query."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a polite and user-friendly support assistant.\n"
     "Use the following KB context to answer the user question clearly:\n\n"
     "Context:\n{context}\n\n"
     "Guidelines:\n"
     "- Always explain in simple and helpful language.\n"
     "- If the KB does not have enough info, say: 'I’m not sure about that. Please contact support.'"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [47]:
# Chains
history_aware_retriever = create_history_aware_retriever(llm2, retriever, retriever_prompt)
doc_chain = create_stuff_documents_chain(llm2, qa_prompt)
retrieval_chain = create_retrieval_chain(history_aware_retriever, doc_chain)

chat_history = ChatMessageHistory()

conversational_chain = RunnableWithMessageHistory(
    retrieval_chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)


In [48]:
# =========================
# 7. Demo Chat
# =========================
def ask_bot(user_id, query):
    result = conversational_chain.invoke(
        {"input": query, "context": ""},
        config={"configurable": {"session_id": user_id}}
    )
    return result["answer"]

print("🤖 Bot:", ask_bot("user123", "How do I update my billing information?"))

print("\n")
print("🤖 Bot:", ask_bot("user123", "what next?"))


🤖 Bot: I'm happy to help you with that. Unfortunately, updating billing information isn't something that's covered in our current knowledge base. 

I’m not sure about that. Please contact support. They'll be able to guide you through the process and ensure that your billing information is updated correctly.


🤖 Bot: If you'd like to contact support, you can try a few options. 

Firstly, you can check our website for a contact page or a support email address where you can send a message.

Alternatively, if you're looking for immediate assistance, you can try our live chat feature (if it's available) or give us a call (if we have a phone number listed).

If none of these options are available, you can try reaching out to our support team via our social media channels.

Remember, our support team is here to help you, and they'll be able to assist you with updating your billing information.
